In [59]:
import requests
import datetime
import wikipedia

from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
from pytz import UTC

wikipedia.set_lang('pt')

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f'{temp_atual}ºC'
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

In [60]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])
chat = ChatOpenAI()

tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

In [61]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [62]:
resposta = chain.invoke({'input': 'Qual a temperatura em Floripa?'})
resposta

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5954, 'longitude': -48.548}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5954, 'longitude': -48.548}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 131, 'total_tokens': 158}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5dfda41e-b897-4e87-a84d-becf44a6c38c-0', usage_metadata={'input_tokens': 131, 'output_tokens': 27, 'total_tokens': 158})])

In [63]:
resposta.tool, resposta.tool_input, resposta.message_log

('retorna_temperatura_atual',
 {'latitude': -27.5954, 'longitude': -48.548},
 [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 131, 'total_tokens': 158}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5dfda41e-b897-4e87-a84d-becf44a6c38c-0', usage_metadata={'input_tokens': 131, 'output_tokens': 27, 'total_tokens': 158})])

In [64]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [65]:
resposta_inicial = chain.invoke({
    'input': 'Qual a temperatura em Floripa?',
    'agent_scratchpad': []})
resposta_inicial

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5954, 'longitude': -48.548}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5954, 'longitude': -48.548}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 131, 'total_tokens': 158}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d862fa70-b899-4537-9e41-9fdbb7077114-0', usage_metadata={'input_tokens': 131, 'output_tokens': 27, 'total_tokens': 158})])

In [66]:
observacao = tool_run[resposta_inicial.tool].run(resposta_inicial.tool_input)
observacao

'15.0ºC'

In [67]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
format_to_openai_function_messages([(resposta_inicial, observacao)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 131, 'total_tokens': 158}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d862fa70-b899-4537-9e41-9fdbb7077114-0', usage_metadata={'input_tokens': 131, 'output_tokens': 27, 'total_tokens': 158}),
 FunctionMessage(content='15.0ºC', name='retorna_temperatura_atual')]

In [69]:
resposta_final = chain.invoke({
	"input":"Qual a temperatura em Floripa?",
	"agent_scratchpad": format_to_openai_function_messages([(resposta_inicial, observacao)])
})

resposta_final

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5954, 'longitude': -48.548}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5954, 'longitude': -48.548}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 153, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-547e3e97-75ad-4575-90f6-b3409417f404-0', usage_metadata={'input_tokens': 153, 'output_tokens': 27, 'total_tokens': 180})])

In [70]:
from langchain.schema.agent import AgentFinish

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'agent_scratchpad': format_to_openai_function_messages(passos_intermediarios)
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))


In [71]:
run_agent("Qual é a temperatura de Floripa?")

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 15.0ºC.'}, log='A temperatura atual em Florianópolis é de 15.0ºC.')

In [72]:
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough


pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)
chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))


In [73]:
pass_through.invoke({"input": "Qual é a temperatura de Floripa?", "intermediate_steps":[]})

{'input': 'Qual é a temperatura de Floripa?',
 'intermediate_steps': [],
 'agent_scratchpad': []}

In [74]:
run_agent("Qual a temperatura de Floripa?")

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 15.0ºC.'}, log='A temperatura atual em Florianópolis é de 15.0ºC.')

In [75]:
prompt = ChatPromptTemplate.from_messages([
	("system", "Você é um assistente amigável chamado Lucas"),
	("user", "{input}"),
	MessagesPlaceholder(variable_name="agent_scratchpad")
])

pass_through = RunnablePassthrough.assign(
	agent_scratchpad = lambda x: format_to_openai_function_messages(x["intermediate_steps"])
)

chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [76]:
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))